In [1]:
import argparse
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from tensorflow.keras import regularizers

# Suppress TensorFlow info messages
tf.get_logger().setLevel('ERROR')

FLAGS = None

def enhanced_preprocessing():
    df = pd.read_csv('HR_Employee_Cleaned.csv')
    
    # Handle missing values if any
    df.dropna(inplace=True)
    
    # Convert categoricals using label encoding where ordinal
    ordinal_features = ['Education', 'JobLevel', 'StockOptionLevel']
    for col in ordinal_features:
        df[col] = df[col].astype('category').cat.codes
    
    # Separate features and target
    X = df.drop('Attrition', axis=1)
    y = df['Attrition']
    
    # Handle class imbalance with SMOTE
    smote = SMOTE(random_state=42)
    X_res, y_res = smote.fit_resample(X, y)
    
    # Robust scaling for numerical features
    scaler = RobustScaler()
    numerical_features = X.select_dtypes(include=np.number).columns
    X_res[numerical_features] = scaler.fit_transform(X_res[numerical_features])
    
    return X_res.values, y_res.values

def build_optimized_model(input_shape, hp):
    model = tf.keras.Sequential()
    
    # Input layer
    model.add(tf.keras.layers.Input(shape=(input_shape,)))
    
    # Hidden layers with dropout and regularization
    for _ in range(hp['num_layers']):
        model.add(tf.keras.layers.Dense(
            hp['hidden_units'],
            activation='relu',
            kernel_regularizer=regularizers.l2(hp['l2_reg'])))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dropout(hp['dropout_rate']))
    
    # Output layer
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    # Custom learning rate schedule
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=hp['learning_rate'],
        decay_steps=1000,
        decay_rate=0.9)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    
    model.compile(optimizer=optimizer,
                loss='binary_crossentropy',
                metrics=['accuracy',
                        tf.keras.metrics.Precision(name='precision'),
                        tf.keras.metrics.Recall(name='recall')])
    return model

def main(argv=None):
    X, y = enhanced_preprocessing()
    
    # Hyperparameter configuration
    hp = {
        'num_layers': FLAGS.num_layers,
        'hidden_units': FLAGS.hidden_units,
        'l2_reg': FLAGS.l2_reg,
        'dropout_rate': FLAGS.dropout_rate,
        'learning_rate': FLAGS.learning_rate,
        'batch_size': FLAGS.batch_size,
        'epochs': FLAGS.epochs
    }
    
    # Cross-validation setup
    skf = StratifiedKFold(n_splits=5, shuffle=True)
    fold = 1
    
    for train_idx, val_idx in skf.split(X, y):
        print(f"\nFold {fold} - {'-'*30}")
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        model = build_optimized_model(X_train.shape[1], hp)
        
        # Early stopping callback
        es = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True)
        
        history = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=hp['epochs'],
            batch_size=hp['batch_size'],
            callbacks=[es],
            verbose=1)
        
        # Evaluate performance
        y_pred = (model.predict(X_val) > 0.5).astype(int)
        print(f"\nFold {fold} Classification Report:")
        print(classification_report(y_val, y_pred))
        
        fold += 1

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--num_layers', type=int, default=3,
                      help='Number of hidden layers')
    parser.add_argument('--hidden_units', type=int, default=64,
                      help='Number of units per hidden layer')
    parser.add_argument('--l2_reg', type=float, default=0.001,
                      help='L2 regularization factor')
    parser.add_argument('--dropout_rate', type=float, default=0.3,
                      help='Dropout rate between layers')
    parser.add_argument('--learning_rate', type=float, default=0.001,
                      help='Initial learning rate')
    parser.add_argument('--batch_size', type=int, default=32,
                      help='Training batch size')
    parser.add_argument('--epochs', type=int, default=100,
                      help='Maximum training epochs')
    
    FLAGS, unparsed = parser.parse_known_args()
    tf.compat.v1.app.run(main=main, argv=[sys.argv[0]] + unparsed)

ModuleNotFoundError: No module named 'imblearn'

In [ ]:
python script.py --num_layers 4 --hidden_units 128 --l2_reg 0.0001 --dropout_rate 0.2 --learning_rate 0.0005 --batch_size 64

In [ ]:
# After running the MLP model code in HR_Project_MLP_Model.ipynb

# 1. Training History Plot
plt.figure(figsize=(12, 5))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

# 2. Confusion Matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(y_val, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Attrition', 'Attrition'], 
            yticklabels=['No Attrition', 'Attrition'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('MLP Confusion Matrix')
plt.show()

# 3. ROC Curve
from sklearn.metrics import roc_curve, auc

y_proba = model.predict(X_val)
fpr, tpr, _ = roc_curve(y_val, y_proba)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('MLP ROC Curve')
plt.legend(loc="lower right")
plt.show()